# Calling Web Application Programming Interfaces (APIs)

Many of the specifications we need (increasingly so) are not totally static.  Relying on a static file for importing specifications leaves us open to: 
* missing out on updates
* inability to link/reuse 


Many of the core resources for Clinical Research standards management are available via the web using Universal Resource Locations (URL).  

In general there is a request-response pattern for accessing web resources; we make a request to the service and get a response back.  Checking the response to make sure the request worked is key

In [ ]:
# import a library for making web requeststs
import requests

# make a simple request

response = requests.get("https://cdisc.org")

# The response object is how you work out whether your request worked. 

## A Web Services Response 

The response object tells us how the request proceeeded; key attributes:
* `status_code` - the status code returned by the web server
* `text` - the text of the response
* `json` - if the request returns a JSON structure, the JSON can be automatically converted 

In [ ]:
# let's look at the response
print(f"Status Code: {response.status_code}")

if response.status_code == 200:
    print("All ok")


In [ ]:
print("Let's look at what the website states it sent to the client")

print(f"Content-type: {response.headers.get('Content-type')}")

print("Content, truncated: ", response.text[:300])

In [ ]:
# What happens when we request a website or page that doesn't exist?

bad_guy = requests.get("https://cdisc.org/say-no-to-xpt")

print(f"Requesting {bad_guy.url} got a status code of {bad_guy.status_code}")

print("404 is the HTTP Status code for Not Found")

In [ ]:
# Always check your return codes 

if not 200 <= bad_guy.status_code <= 300:
    print("Request failed")


In [ ]:
# Let's try and pull a more prevalent data transfer format, namely FHIR

valuesets = requests.get("https://fhir.nhs.uk/ValueSet")


In [ ]:
# lets inspect the valuesets response
print(f"Requesting {valuesets.url} got a status of {valuesets.status_code}")
print(f"Content type is {valuesets.headers.get('Content-type')}")

In [ ]:
print("Response content (truncated)", valuesets.text[:300])

In [ ]:
# XML is a popular exchange format, but it's not too fun to process -> let's request a JSON response

# the request has a set of Headers that communicate directly to the Server, the set of headers is stand
valuesets_json = requests.get("https://fhir.nhs.uk/ValueSet", headers={"Accept": "application/fhir+json"})

In [ ]:
# let's inspect the valuesets response
print(f"Requesting {valuesets_json.url} got a status of {valuesets_json.status_code}")
print(f"Content type is {valuesets_json.headers.get('Content-type')}")

In [ ]:
# We can access the data as a python dict

valueset_dict = valuesets_json.json()
print(f"Keys: {valueset_dict.keys()}")

In [ ]:
print(f"ValueSet has {len(valueset_dict.get('entry'))} values")

In [ ]:
for vs in valueset_dict.get('entry'):
    resource = vs.get('resource')
    print(f"{resource.get('id')} -> {resource.get('status')}")


Now we know how to query web APIs, how to anticipate and handle errors and how to access the data.  Let's move on to a relevant web API to look at querying this using Python